In [1]:
import requests
import json
import pandas as pd
import os
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.translate import AlignedSent, Alignment
import goslate
import time
from translate import Translator
from difflib import SequenceMatcher
from collections import defaultdict

In [ ]:
# These are done only ones
#nltk.download('punkt')
#nltk.download('stopwords')

In [4]:
# Read all the MAG and Institutions
MAG_DICT = None
with open("../data/gen/dh_in_mag.json") as json_file:
    MAG_DICT = json.load(json_file)

ACDH_INST_PATH = "../data/acdh/institutions.json"
with open(ACDH_INST_PATH) as json_file:
    acdh_inst = json.load(json_file)

In [ ]:
# all the unique institutions we have 
# Some values should ba manually corrected 
# For instance: 'Consiglio Nazionale delle Ricerche'  _and_ 'Consiglio Nazionale delle Ricerche - CNR' are the same institution
# This goes out of this work

inst_df = pd.DataFrame(acdh_inst)
inst_df = inst_df[["name","id"]]
inst_df.columns = ['acdh_name','id']
inst_df['acdh_name'] = inst_df['acdh_name'].apply(lambda x: x.strip().lower()) 
#inst_df['language'] = inst_df['acdh_name'].apply(lambda x: gs.detect(x))
inst_df['acdh_name_eng'] = inst_df['acdh_name'].apply(lambda x: Translator(from_lang = "autodetect", to_lang="en").translate(x).lower())
inst_df['acdh_name_eng'] = inst_df['acdh_name_eng'].apply(lambda x: '' if 'please select' in x else x)
#inst_df['acdh_tokenized'] = inst_df['acdh_name'].apply(lambda x: list(set(word_tokenize(x)) - set(stopwords.words('english'))) )

In [2]:
ACDH_INST_TRANSLATION = "../data/gen/inst_translation.json"
acdh_inst_translation = None
with open(ACDH_INST_TRANSLATION) as json_file:
    acdh_inst_translation = json.load(json_file)

In [5]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

for art_index in range(0,len(MAG_DICT)):
    art = MAG_DICT[art_index]
    l_aff = []
    for aff in art['AA.DAfN']:  
        # Check inside the ACDH affiliations if present
        flag_found = -1
        for inst in acdh_inst_translation:
            if ((similar(inst["acdh_name"],aff) > 0.8) or (similar(inst["acdh_name_eng"],aff) > 0.8)):
                flag_found = inst["id"]
                break
      
        l_aff.append(flag_found)
    MAG_DICT[art_index]["acdh_aff"] = l_aff

In [6]:
EXPORT_READY_MAG_DICT = {}
for item in MAG_DICT:
    EXPORT_READY_MAG_DICT[item["DOI"]] = {}
    EXPORT_READY_MAG_DICT[item["DOI"]]["AA.DAfN"] = item["AA.DAfN"]
    EXPORT_READY_MAG_DICT[item["DOI"]]["DOI"] = item["DOI"]
    EXPORT_READY_MAG_DICT[item["DOI"]]["acdh_aff"] = item["acdh_aff"]

In [8]:
with open("../data/res/affiliation_v2.json", 'w') as fp:
    json.dump(EXPORT_READY_MAG_DICT, fp)

In [ ]:
#An error has been detected regarding ... 
with open("../data/res/affiliation.json") as json_file:
    affiliation_df = pd.DataFrame(json.load(json_file))

ERRORS = [" Pisa"]
affiliation_df = affiliation_df.set_index('DOI').drop(" Pisa")
affiliation_df.to_json("../data/res/affiliation(with_id).json", orient='index')